In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
base_dir = 'cats_dogs_small'
train_dir = 'cats_dogs_small/train'
test_dir = 'cats_dogs_small/test'
val_dir = 'cats_dogs_small/validation'

In [3]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape =(150,150,3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [5]:
print('Количество тренировочных весов перед замороской:', len(model.trainable_weights))

Количество тренировочных весов перед замороской: 30


In [6]:
conv_base.trainable = False

In [7]:
print('Количество тренировочных весов после замороской:', len(model.trainable_weights))

Количество тренировочных весов после замороской: 4


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')
val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.Adam(lr=2e-5),
             metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=val_generator,
    validation_steps=50)

model.save(cats_and_dogs_v1.h5)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 122s 1s/step - loss: 0.7067 - acc: 0.5585 - val_loss: 0.5743 - val_acc: 0.7400
Epoch 2/30
100/100 [==============================] - 94s 944ms/step - loss: 0.5890 - acc: 0.6750 - val_loss: 0.5067 - val_acc: 0.7740
Epoch 3/30
100/100 [==============================] - 95s 946ms/step - loss: 0.5342 - acc: 0.7280 - val_loss: 0.4759 - val_acc: 0.7750
Epoch 4/30
100/100 [==============================] - 95s 947ms/step - loss: 0.4886 - acc: 0.7705 - val_loss: 0.4539 - val_acc: 0.7900
Epoch 5/30
100/100 [==============================] - 95s 946ms/step - loss: 0.4634 - acc: 0.7725 - val_loss: 0.4240 - val_acc: 0.8110
Epoch 6/30
100/100 [==============================] - 95s 947ms/step - loss: 0.4430 - acc: 0.7905 - val_loss: 0.4136 - val_acc: 0.8250
Epoch 7/30
100/100 [==============================] - 95s 947ms/step - loss: 0.4386 - acc: 0.8060 - val_loss

NameError: name 'cats_and_dogs_v1' is not defined

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.figure()
plt.plot(epochs, acc, 'bo', label='Точность на обучении')
plt.plot(epochs, val_acc, 'b', label='Точность на проверке')
plt.title('Обучающая и проверочная точность')
plt.legend()
plt.show()